# Code preparation

In [45]:
import json
import numpy as np
from datetime import datetime, timedelta
import torch
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from utils.utils import load_data, entropy

In [2]:
# Parameterize time (in seconds) onto the unit circle 
def time_to_circle(time):
    seconds_per_day = 60*60*24
    cos_time = np.cos(2*np.pi*time/seconds_per_day)
    sin_time = np.sin(2*np.pi*time/seconds_per_day)
    return cos_time, sin_time

In [3]:
# Converting Unix time onto the unit circle (time of day) and the week day
def time_stamp(times):
    datetimes = np.array([datetime.fromtimestamp(time[1]) for time in times])
    week_day = [time.weekday() for time in datetimes]
    times = np.array([time.second + time.minute*60 + time.hour*60*60 for time in datetimes])
    time_circle = time_to_circle(times)
    return time_circle, week_day
    

In [4]:
# Creates list of zeros and ones where ones means that it is the first time the location is seen.
def explore(path):
    seen = set()
    explore_path = []
    for place in path:
        if place in seen:
            explore_path.append(0)
        else:
            explore_path.append(1)
            seen.add(place)
    return explore_path

In [5]:
# Preprocess all the data in saved files such that 1 file contains all the information for one person.
# A file for test, train and all of the data is created.
# In total it created a file containing the matrix: [path,time_cos,time_sin,week_day,explore_path,person_ID,lat,lon]
# It also relables the location id in path such that the amount if descending the higher the id.
def prepare_data(peps):
    tmp_max = 0
    for pep in peps:
        try:
            with open(f"Data/{pep}/final_series.json") as json_file:
                data_path = json.load(json_file)
            with open(f"Data/{pep}/timestamps.json") as json_file:
                time = json.load(json_file)
            with open(f"Data_m_loc/{pep}/stop_coords.json") as json_file:
                data_loc = np.array(json.load(json_file))

            [time_cos,time_sin], week_day = time_stamp(time)

            data_path = [path[-1] for path in data_path]
            
            #data_dist = [distance(coord_1,coord_2).km/1000 for coord_1,coord_2 in zip(data_loc[data_path[:-1]],data_loc[data_path[1:]])]

            explore_path = explore(data_path)
            person_list = [pep for _ in range(len(explore_path))]
            
            data = np.array([data_path,time_cos,time_sin,week_day,explore_path,person_list,data_loc.transpose()[0][data_path],data_loc.transpose()[1][data_path]])
            
            frac = int(data.shape[1]*0.90)
            data_train = data[:,:frac].copy()
            data_test = data[:,frac:].copy()
            np.save(f"Data/{pep}/prepared_data_train.npy",data_train)
            np.save(f"Data/{pep}/prepared_data_test.npy",data_test)
            np.save(f"Data/{pep}/prepared_data_all.npy",data)
            
            
            counter = Counter(data_train[0,:])
            counter_all = Counter(data[0,:])
            ranks = {rank[0]:i for i,rank in enumerate(counter.most_common())}
            ranks_all = {rank[0]:i for i,rank in enumerate(counter_all.most_common())}
            data_train_relabeled = [ranks[loc] for loc in data_train[0,:]]
            data_test_relabeled = [ranks[loc] if loc in ranks else 900 for loc in data_test[0,:] ]
            data_all_relabeled = [ranks_all[loc] for loc in data[0,:]]
            data_train[0,:] = data_train_relabeled
            data_test[0,:] = data_test_relabeled
            data[0,:] = data_all_relabeled
            data_location_20 = [data_loc[int(loc[0])] for loc in counter.most_common(100)]
            np.save(f"Data/{pep}/prepared_data_train_relabeled.npy",data_train)
            np.save(f"Data/{pep}/prepared_data_test_relabeled.npy",data_test)
            np.save(f"Data/{pep}/prepared_data_all_relabeled.npy",data)
            np.save(f"Data/{pep}/prepared_data_locations_relabeled.npy",data_location_20)
            df_ranks = pd.DataFrame(list(ranks.keys()),list(ranks.values()))
            df_ranks.to_pickle(f"Data/{pep}/label_dict.pkl")
            
        except FileNotFoundError:
            print(f"File {pep} not found")
            continue

In [37]:
# Preprocess all the data in saved files such that 1 file contains all the information for one person.
# A file for test, train and all of the data is created.
# In total it created a file containing the matrix: [path,time_cos,time_sin,week_day,explore_path,person_ID,lat,lon]
# It also relables the location id in path such that the amount if descending the higher the id.
def prepare_data_global(peps):
    for pep in tqdm(peps):
        try:
            data_tmp = np.genfromtxt(f"data/trace{pep}.csv",delimiter=',',skip_header=True,dtype="int,int,int", names = "loc,start,end")
            if data_tmp.shape[0]==0:
                continue
            with open(f"data/label_coords.json") as json_file:
                dict_loc = json.load(json_file)
            #Removes -1 and and dublicates
            data_tmp = data_tmp[data_tmp['loc'] >= 0]
            data_tmp = np.array([v for i, v in enumerate(data_tmp) if i == 0 or v[0] != data_tmp[i-1][0]])


            [time_cos,time_sin], week_day = time_stamp(data_tmp[['start','end']])
            data_path = data_tmp['loc']
            data_loc = np.array([dict_loc[str(loc)] for loc in data_path])
            #data_dist = [distance(coord_1,coord_2).km/1000 for coord_1,coord_2 in zip(data_loc[data_path[:-1]],data_loc[data_path[1:]])]


            explore_path = explore(data_path)

            person_list = [pep for _ in range(len(explore_path))]

            #data = np.array([data_path,time_cos,time_sin,week_day,explore_path,person_list,data_loc.transpose()[0][data_path],data_loc.transpose()[1][data_path]],data_path)
            data = np.array([data_path,time_cos,time_sin,week_day,explore_path,person_list,data_loc.transpose()[0],data_loc.transpose()[1],data_path])

            frac = int(data.shape[1]*0.90)
            data_train = data[:,:frac].copy()
            data_test = data[:,frac:].copy()

            counter = Counter(data_train[0,:])
            counter_all = Counter(data[0,:])
            ranks = {rank[0]:i for i,rank in enumerate(counter.most_common())}
            ranks_all = {rank[0]:i for i,rank in enumerate(counter_all.most_common())}
            data_train_relabeled = [ranks[loc] for loc in data_train[0,:]]
            data_test_relabeled = [ranks[loc] if loc in ranks else 900 for loc in data_test[0,:] ]
            data_all_relabeled = [ranks_all[loc] for loc in data[0,:]]
            data_train[0,:] = data_train_relabeled
            data_test[0,:] = data_test_relabeled
            data[0,:] = data_all_relabeled
            
            
            
            data_location_20 = [dict_loc[str(int(loc[0]))] for loc in counter.most_common(100)]
            os.makedirs(f"data/{pep}/", exist_ok=True)
            np.save(f"data/{pep}/prepared_data_train_relabeled.npy",data_train)
            np.save(f"data/{pep}/prepared_data_test_relabeled.npy",data_test)
            np.save(f"data/{pep}/prepared_data_all_relabeled.npy",data)
            np.save(f"data/{pep}/prepared_data_locations_relabeled.npy",data_location_20)
            df_ranks = pd.DataFrame(list(ranks.keys()),list(ranks.values()))
            df_ranks.to_pickle(f"data/{pep}/label_dict.pkl")
            
        except (FileNotFoundError, OSError):
            print(f"File {pep} not found")
            continue

In [27]:
# Saves some meta data of all users
def prepare_user_inf(peps):
    df = pd.DataFrame(columns = ["length"])
    for pep in peps:
        try:
            data_tmp = np.genfromtxt(f"data/trace{pep}.csv",delimiter=',',skip_header=True,dtype="int,int,int", names = "loc,start,end")
            if data_tmp.shape[0]==0:
                print(f"File {pep} was empty")
                continue
            data_tmp = data_tmp[data_tmp['loc'] >= 0]
            data_tmp = np.array([v for i, v in enumerate(data_tmp) if i == 0 or v[0] != data_tmp[i-1][0]])
            data_path = data_tmp['loc']
            #with open(f"data/labels_infomap_corr03.json") as json_file:
            #    data_cluster = json.load(json_file)
                
            df.loc[pep] = [len(data_path)] #, data_cluster[str(pep)]]
        except OSError:
            print(f"File {pep} not found")
    #return df
    df.to_pickle("data_inf.pkl")

In [26]:
prepare_user_inf(range(856))

D:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: genfromtxt: Empty input file: "data/trace821.csv"
  


File 821 was empty


D:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: genfromtxt: Empty input file: "data/trace846.csv"
  


File 846 was empty
File 852 not found
File 853 not found
File 854 not found
File 855 not found


In [38]:
prepare_data_global(range(856))


 11%|██████████████▋                                                                                                                           | 91/856 [00:06<01:00, 12.66it/s]


 21%|█████████████████████████████▍                                                                                                           | 184/856 [00:14<00:54, 12.32it/s]


 32%|████████████████████████████████████████████▏                                                                                            | 276/856 [00:21<00:44, 13.16it/s]


 43%|███████████████████████████████████████████████████████████▍                                                                             | 371/856 [00:28<00:35, 13.85it/s]


 54%|██████████████████████████████████████████████████████████████████████████                                                               | 463/856 [00:36<00:30, 13.05it/s]


...

 76%|███████████████████████████████████████████████████████████████████████████████████████████████████████▋                                 | 648/856 [00:50<00:16, 12.82it/s]


 88%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                 | 750/856 [00:57<00:06, 17.49it/s]


 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍     | 821/856 [01:01<00:02, 16.36it/s]D:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: genfromtxt: Empty input file: "data/trace821.csv"
  

 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 845/856 [01:02<00:00, 15.15it/s]D:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:8: UserWarning: genfromtxt: Empty input file: "data/trace846.csv"
  

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 852/856 [01:03<00:00, 16.70it/s]

File 852 not found
File 853 not found
File 854 not found
File 855 not found


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 856/856 [01:03<00:00, 13.52it/s]


In [46]:
entropy_dict = {}
step = 10
for pep in tqdm(range(856)):
    if str(pep) not in entropy_dict:
        data = load_data([pep],bs=1,dat_type="all_relabeled",stop_t=False)[0]
        data = data.t().reshape(-1).cpu().numpy()[1:]
        entropy_dict[pep] = []
        for ix in range(step,len(data),step):
            entropy_dict[pep].append(entropy(list(data[:ix]),tqdm_boo=True)[0])


  5%|███████▍                                                                                                                                | 47/856 [03:42<1:01:01,  4.53s/it]


 11%|███████████████▏                                                                                                                          | 94/856 [07:41<26:44,  2.11s/it]


 17%|██████████████████████▋                                                                                                                | 144/856 [13:35<3:09:11, 15.94s/it]


 22%|██████████████████████████████▋                                                                                                          | 192/856 [17:27<20:22,  1.84s/it]


 28%|█████████████████████████████████████▋                                                                                                 | 239/856 [21:41<1:15:55,  7.38s/it]


...


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 846/856 [1:13:56<00:08,  1.13it/s]

File 846 not found



 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 851/856 [1:13:56<00:02,  2.08it/s]

File 852 not found
File 853 not found
File 854 not found
File 855 not found


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 856/856 [1:13:56<00:00,  5.18s/it]


In [47]:
good_peps = []
for i,(pep,ent) in enumerate(entropy_dict.items()):
    y_slope = np.array([abs(ent[j]-ent[j+5]) for j in range(len(ent)-5)])
    for k in range(len(y_slope)-15):
        if sum(abs(y_slope[k:k+10]-y_slope[k+5:k+15])<0.05)>=8:
            good_peps.append(pep)
            break

In [48]:
np.savetxt("good_peps.txt",np.array(good_peps,int))